In [8]:
from data_utils import get_data_loaders, prepare_train, get_cv_data_loaders, prepare_test
from engine import training_step, validation_step
from utils import print_trainable_parameters
from model import ClsModel1, save_model, load_model

import pandas as pd
import torch
from torch import nn

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
model  = ClsModel1(input_size=21, hidden_size=100,num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

trainable params: 103,301 || all params: 103,301 || trainable%: 100.0


In [10]:
df=pd.read_csv("data/train.csv", index_col='id')
X_train, y_train, scaler = prepare_train(df)
test = pd.read_csv("data/test.csv", index_col='id')
test = prepare_test(test, scaler)

In [11]:
import os

def predict_with_models(test, directory ,model, optimizer):
    predictions = {}
    model_files = [f for f in os.listdir(directory) if f.endswith('.pt')]
    model_files.sort()
    for model_file in model_files:
        model_name = os.path.splitext(model_file)[0]
        model, optimizer, epoch, loss, scheduler  = load_model(model,optimizer, os.path.join(directory, model_file))
        model = model.to(device)
        with torch.inference_mode():
            preds = model(torch.tensor(test,device=device,dtype=torch.float64))
            preds = preds.cpu().detach().numpy()
        predictions[model_name] = preds.squeeze()
    return pd.DataFrame(predictions)

In [ ]:
preds_df = predict_with_models(test, 'saved_models', model, optimizer)

In [ ]:
preds_df

,model_0,model_1,model_2,model_3,model_4
0,0.625716,0.629981,0.671341,0.601747,0.633632
1,0.281486,0.255844,0.313442,0.292592,0.277122
2,0.378115,0.286368,0.306559,0.379653,0.386118
3,0.020716,0.021280,0.020792,0.017670,0.016126
4,0.647994,0.620876,0.645687,0.649672,0.654906
...,...,...,...,...,...
106166,0.574141,0.592322,0.573035,0.502824,0.590881
106167,0.705952,0.706302,0.727500,0.719100,0.702863
106168,0.408289,0.325725,0.411207,0.406743,0.335154
106169,0.064741,0.076330,0.091181,0.057667,0.054771


In [13]:
from model import ClsModel2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)
model  = ClsModel2(input_size=21, hidden_size=50,num_classes=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
with torch.inference_mode():
            preds = model(torch.tensor(test,device=device,dtype=torch.float64))
            preds = preds.cpu().detach().numpy()
preds

trainable params: 16,901 || all params: 16,901 || trainable%: 100.0


array([[0.5694776 ],
       [0.5843513 ],
       [0.69464756],
       ...,
       [0.40443002],
       [0.34022056],
       [0.43700609]])

In [15]:
submission = pd.read_csv("data/sample_submission.csv", index_col='id')
submission['smoking'] = preds
submission.to_csv('data/submission.csv')

In [16]:
submission.isna().sum()

smoking    0
dtype: int64